In [19]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn import datasets
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.stats import norm

from Average_Correlation import Calculate_Average_Correlation
from Distance_Matrix import Translate_To_Distance_Matrix

Calculate Average Correlation of each clusters

In [20]:
# log_mv2, log_pr2, log_dj2, raw_mv2, raw_pr2, raw_dj2 총 6개의 파일을 넣어서 실행 가능
data = pd.read_csv('../data/log_mv2.csv', index_col = 0)
data

,SPDV,EEMD,HDGE,DWSH,CWS,AADR,HYLD,MINC,FLRT,DBLV,...,EMCB,HYDW,HYUP,SHYL,price_mv_boostvc_portfolio,price_mv_dcg_portfolio,price_mv_electric_capital_portfolio,price_mv_huobi_capital_portfolio,price_mv_1confirmation_portfolio,price_mv_placeholder_ventures_portfolio
2019-01-02,0.001564,0.012645,-0.003565,-0.013503,0.005248,0.006468,0.000594,-0.001269,-0.000511,0.010197,...,0.003230,-0.000738,-0.000045,0.000276,0.035010,0.035508,0.034948,0.035415,0.035450,0.035290
2019-01-03,-0.011853,-0.004027,0.022367,0.007926,-0.025664,-0.011221,-0.000891,0.001269,0.000383,-0.007979,...,0.001026,-0.002303,-0.001977,-0.001595,-0.025301,-0.025144,-0.025347,-0.025428,-0.025595,-0.025571
2019-01-04,0.027401,0.028125,-0.039174,-0.049356,0.021775,0.038130,0.007992,0.001056,-0.000170,0.028729,...,-0.001026,0.011003,0.021040,0.014346,0.000218,0.000024,0.000250,0.000212,0.000126,0.000292
2019-01-07,0.010177,-0.001489,-0.019560,-0.023639,0.010557,0.011758,0.009390,0.001265,0.009433,0.005286,...,0.009628,0.006610,0.000000,0.005774,0.052947,0.053579,0.052878,0.052884,0.053124,0.053034
2019-01-08,0.010197,0.004281,-0.008680,-0.018212,0.001273,0.006894,0.003499,0.000421,0.009471,0.003290,...,-0.002762,0.005534,0.000000,0.004889,0.002953,0.002954,0.003025,0.002972,0.002980,0.003126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.011844,0.013797,-0.002820,0.000000,0.009259,0.005959,0.000314,-0.000207,0.000103,0.009611,...,-0.006080,0.000790,0.001674,0.001370,0.000052,0.000338,0.000597,0.000319,0.000725,0.000726
2021-12-28,0.003391,-0.000315,-0.000807,-0.001170,0.003918,0.006545,0.001447,0.001242,0.000513,0.001932,...,0.004057,-0.000099,-0.001884,-0.000316,-0.062169,-0.062073,-0.062531,-0.062542,-0.062522,-0.062641
2021-12-29,0.003294,-0.010028,0.002822,-0.005283,-0.004069,-0.003891,-0.005675,-0.000207,0.000410,0.002333,...,-0.001621,-0.001581,-0.001677,-0.000737,-0.018749,-0.018688,-0.018837,-0.018664,-0.018619,-0.018581
2021-12-30,-0.003391,-0.001428,-0.008084,-0.004129,0.007803,-0.000624,0.001264,0.000000,0.000103,-0.004061,...,0.002700,-0.000376,-0.001155,-0.000422,0.013593,0.013704,0.013570,0.013847,0.013684,0.013643


In [21]:
data = data.replace(0,np.NaN)
for i in data.columns:
    df1 = data[i]
    if df1.isna().sum() >= 100:
        data = data.drop(i, axis = 1)
data = data.fillna(0)
data

,SPDV,EEMD,HDGE,DWSH,CWS,AADR,HYLD,MINC,FLRT,DBLV,...,DWMF,HYDW,HYUP,SHYL,price_mv_boostvc_portfolio,price_mv_dcg_portfolio,price_mv_electric_capital_portfolio,price_mv_huobi_capital_portfolio,price_mv_1confirmation_portfolio,price_mv_placeholder_ventures_portfolio
2019-01-02,0.001564,0.012645,-0.003565,-0.013503,0.005248,0.006468,0.000594,-0.001269,-0.000511,0.010197,...,-0.000441,-0.000738,-0.000045,0.000276,0.035010,0.035508,0.034948,0.035415,0.035450,0.035290
2019-01-03,-0.011853,-0.004027,0.022367,0.007926,-0.025664,-0.011221,-0.000891,0.001269,0.000383,-0.007979,...,-0.009311,-0.002303,-0.001977,-0.001595,-0.025301,-0.025144,-0.025347,-0.025428,-0.025595,-0.025571
2019-01-04,0.027401,0.028125,-0.039174,-0.049356,0.021775,0.038130,0.007992,0.001056,-0.000170,0.028729,...,0.028109,0.011003,0.021040,0.014346,0.000218,0.000024,0.000250,0.000212,0.000126,0.000292
2019-01-07,0.010177,-0.001489,-0.019560,-0.023639,0.010557,0.011758,0.009390,0.001265,0.009433,0.005286,...,0.000866,0.006610,0.000000,0.005774,0.052947,0.053579,0.052878,0.052884,0.053124,0.053034
2019-01-08,0.010197,0.004281,-0.008680,-0.018212,0.001273,0.006894,0.003499,0.000421,0.009471,0.003290,...,0.007329,0.005534,0.000000,0.004889,0.002953,0.002954,0.003025,0.002972,0.002980,0.003126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,0.011844,0.013797,-0.002820,0.000000,0.009259,0.005959,0.000314,-0.000207,0.000103,0.009611,...,-0.002627,0.000790,0.001674,0.001370,0.000052,0.000338,0.000597,0.000319,0.000725,0.000726
2021-12-28,0.003391,-0.000315,-0.000807,-0.001170,0.003918,0.006545,0.001447,0.001242,0.000513,0.001932,...,0.003002,-0.000099,-0.001884,-0.000316,-0.062169,-0.062073,-0.062531,-0.062542,-0.062522,-0.062641
2021-12-29,0.003294,-0.010028,0.002822,-0.005283,-0.004069,-0.003891,-0.005675,-0.000207,0.000410,0.002333,...,0.001123,-0.001581,-0.001677,-0.000737,-0.018749,-0.018688,-0.018837,-0.018664,-0.018619,-0.018581
2021-12-30,-0.003391,-0.001428,-0.008084,-0.004129,0.007803,-0.000624,0.001264,0.000000,0.000103,-0.004061,...,-0.004501,-0.000376,-0.001155,-0.000422,0.013593,0.013704,0.013570,0.013847,0.013684,0.013643


In [22]:
distance_matrix = Translate_To_Distance_Matrix(data.corr())
distance_matrix

,SPDV,EEMD,HDGE,DWSH,CWS,AADR,HYLD,MINC,FLRT,DBLV,...,DWMF,HYDW,HYUP,SHYL,price_mv_boostvc_portfolio,price_mv_dcg_portfolio,price_mv_electric_capital_portfolio,price_mv_huobi_capital_portfolio,price_mv_1confirmation_portfolio,price_mv_placeholder_ventures_portfolio
SPDV,0.000000,0.363821,0.958889,0.958974,0.326714,0.418888,0.516625,0.631240,0.663852,0.214071,...,0.306771,0.373854,0.380692,0.354533,0.627541,0.626852,0.626803,0.626672,0.626816,0.626811
EEMD,0.363821,0.000000,0.916099,0.904893,0.403292,0.350148,0.541294,0.653778,0.658351,0.357455,...,0.313237,0.401038,0.403964,0.397508,0.613636,0.613085,0.613000,0.612860,0.613024,0.613013
HDGE,0.958889,0.916099,0.000000,0.243249,0.916517,0.884685,0.826843,0.744280,0.725594,0.947914,...,0.927996,0.895099,0.900867,0.904897,0.765422,0.765999,0.766052,0.766147,0.766056,0.766048
DWSH,0.958974,0.904893,0.243249,0.000000,0.900758,0.860965,0.828670,0.764636,0.720027,0.939857,...,0.905729,0.886082,0.892159,0.898571,0.764043,0.764557,0.764574,0.764667,0.764587,0.764577
CWS,0.326714,0.403292,0.916517,0.900758,0.000000,0.377242,0.527211,0.625457,0.678413,0.293102,...,0.315086,0.391958,0.400214,0.381167,0.628471,0.627938,0.627863,0.627711,0.627846,0.627861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
price_mv_dcg_portfolio,0.626852,0.613085,0.765999,0.764557,0.627938,0.579337,0.659563,0.703050,0.711847,0.632357,...,0.609444,0.635280,0.643192,0.637170,0.004852,0.000000,0.004698,0.004972,0.004888,0.004585
price_mv_electric_capital_portfolio,0.626803,0.613000,0.766052,0.764574,0.627863,0.579271,0.659637,0.702952,0.711954,0.632324,...,0.609420,0.635357,0.643245,0.637213,0.005076,0.004698,0.000000,0.004027,0.002754,0.002181
price_mv_huobi_capital_portfolio,0.626672,0.612860,0.766147,0.764667,0.627711,0.579185,0.659503,0.702997,0.711865,0.632204,...,0.609371,0.635199,0.643120,0.637094,0.005999,0.004972,0.004027,0.000000,0.003596,0.003463
price_mv_1confirmation_portfolio,0.626816,0.613024,0.766056,0.764587,0.627846,0.579290,0.659617,0.703056,0.711936,0.632315,...,0.609483,0.635311,0.643220,0.637203,0.006128,0.004888,0.002754,0.003596,0.000000,0.001624


In [23]:
from sklearn.cluster import KMeans

cluster_num = 120
model = KMeans(n_clusters = cluster_num)
model.fit(distance_matrix)
cluster_number = model.predict(distance_matrix)

In [24]:
AC_list = Calculate_Average_Correlation(data, cluster_number)
np.mean(AC_list)

0.9082663608650905

Calculating silhouette_coefficient

In [25]:
silhouette = sklearn.metrics.silhouette_samples(distance_matrix, cluster_number)

distance_matrix_copy = distance_matrix
distance_matrix_copy['cluster'] = cluster_number
distance_matrix_copy['silhouette_coeff'] = silhouette
distance_matrix_copy.groupby('cluster')['silhouette_coeff'].mean()

silhouette_score = sklearn.metrics.silhouette_score(distance_matrix_copy, cluster_number)
silhouette_score

0.6713401522905628

In [31]:
distance_matrix_copy

,SPDV,EEMD,HDGE,DWSH,CWS,AADR,HYLD,MINC,FLRT,DBLV,...,HYUP,SHYL,price_mv_boostvc_portfolio,price_mv_dcg_portfolio,price_mv_electric_capital_portfolio,price_mv_huobi_capital_portfolio,price_mv_1confirmation_portfolio,price_mv_placeholder_ventures_portfolio,cluster,silhouette_coeff
SPDV,0.000000,0.363821,0.958889,0.958974,0.326714,0.418888,0.516625,0.631240,0.663852,0.214071,...,0.380692,0.354533,0.627541,0.626852,0.626803,0.626672,0.626816,0.626811,30,0.253093
EEMD,0.363821,0.000000,0.916099,0.904893,0.403292,0.350148,0.541294,0.653778,0.658351,0.357455,...,0.403964,0.397508,0.613636,0.613085,0.613000,0.612860,0.613024,0.613013,68,0.151737
HDGE,0.958889,0.916099,0.000000,0.243249,0.916517,0.884685,0.826843,0.744280,0.725594,0.947914,...,0.900867,0.904897,0.765422,0.765999,0.766052,0.766147,0.766056,0.766048,57,0.060181
DWSH,0.958974,0.904893,0.243249,0.000000,0.900758,0.860965,0.828670,0.764636,0.720027,0.939857,...,0.892159,0.898571,0.764043,0.764557,0.764574,0.764667,0.764587,0.764577,57,0.330783
CWS,0.326714,0.403292,0.916517,0.900758,0.000000,0.377242,0.527211,0.625457,0.678413,0.293102,...,0.400214,0.381167,0.628471,0.627938,0.627863,0.627711,0.627846,0.627861,75,0.053728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
price_mv_dcg_portfolio,0.626852,0.613085,0.765999,0.764557,0.627938,0.579337,0.659563,0.703050,0.711847,0.632357,...,0.643192,0.637170,0.004852,0.000000,0.004698,0.004972,0.004888,0.004585,58,0.994784
price_mv_electric_capital_portfolio,0.626803,0.613000,0.766052,0.764574,0.627863,0.579271,0.659637,0.702952,0.711954,0.632324,...,0.643245,0.637213,0.005076,0.004698,0.000000,0.004027,0.002754,0.002181,58,0.995606
price_mv_huobi_capital_portfolio,0.626672,0.612860,0.766147,0.764667,0.627711,0.579185,0.659503,0.702997,0.711865,0.632204,...,0.643120,0.637094,0.005999,0.004972,0.004027,0.000000,0.003596,0.003463,58,0.994430
price_mv_1confirmation_portfolio,0.626816,0.613024,0.766056,0.764587,0.627846,0.579290,0.659617,0.703056,0.711936,0.632315,...,0.643220,0.637203,0.006128,0.004888,0.002754,0.003596,0.000000,0.001624,58,0.995624


In [33]:
cluster = distance_matrix_copy['cluster']
data = data.transpose()
data['cluster'] = cluster
data = data.transpose()
data.to_csv('kmeans.csv')